<a href="https://colab.research.google.com/github/aeseraiah/DeepLabCut-1/blob/master/API_Microservice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/API_Microservice/testing_AI_Prediction-Microservice/
#os.environ['KAGGLE_CONFIG_DIR'] = data_path

[Errno 2] No such file or directory: '/content/gdrive/MyDrive/API_Microservice/testing_AI_Prediction-Microservice/'
/content


In [ ]:
!pip install fuzzywuzzy
import pandas as pd
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from keras.models import Sequential
import matplotlib.pyplot as plt
import seaborn as sns
from keras.layers import Dense, Input, Dropout
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from joblib import dump
from joblib import load
from tensorflow.keras.models import load_model
import json
from flask import request
from src.load_input_data import *
import os

In [ ]:
data = pd.read_csv('inmate_history.csv') # data = dataframe = data structure in pandas library that is like a table or spreadsheet, where data is organzied into rows and columns
columns = data.columns # columns = a pandas Index object = column labels of a dataframe
cols = data.columns.tolist() # cols = a list of column labels (strings) from the 'data' dataframe

features = data.drop('Suicide Committed', axis=1)
features = features.columns.tolist()
features = ','.join(features) #user input comes in the format of a string

labels = data['Suicide Committed'] # labels = a series object
labels = data[['Suicide Committed']] # labels = a new dataframe with a single column
labels = labels.columns.tolist()
labels = ','.join(labels)

#strip() = removes leading and trailing whitespace characters from a string. "   example feature  " --> "example feature"
#split(',') = splits a string (based on the the comma delimiter) into a list of substrings

features = '  Meals Missed, Violent Offenses, Medication Refused, Depression, Days of Social '
labels = 'Suicide Committed'
feature_names = features.strip().split(',')
label_names = labels.strip().split(',')

print(data)

    Meals Missed (last 30 days)  Violent Offenses (last 30 days)  \
0                            10                               10   
1                            28                               28   
2                            25                               27   
3                             0                                9   
4                            17                               23   
..                          ...                              ...   
95                           13                                1   
96                           30                               29   
97                           26                               27   
98                           21                               28   
99                            1                                7   

    Medication Refused (last 30 days) History of Depression  \
0                                  12                   yes   
1                                  10                    

In [ ]:
# process is a module and extract is a function from the fuzzywuzzy, a library that finds the similiary betwene strings
# selects the best match between a string (input_str) and a list of strings (choices)
# scorer = the scoring algorithm to use for matching strogs
# matches = a list a tuples, each tuple containing two elements: the matches string and its corresponding matching score

input_str = 'example'
choices = ['test1', 'example1', 'testmple']
matches = process.extract(input_str, choices, scorer=fuzz.token_sort_ratio)
# finds the best match from the 'matches' list using the 'max' function
# It does this by comparing the second element of each tuple (the match score)

# lambda x: x[1] --> for each element (tuple) 'x' in the matches list, the lambda function will be applied and te value at index 1 of x will be extracted
# In the lambda function, x is the input paramter and x[1] is the expression evaluated and returned
# best match will return the tuple with the best match score
best_match = max(matches, key=lambda x: x[1])

if best_match[1] >= 30:
  print(best_match[0])
else:
  print('no good match found')


In [ ]:
def feature_label_match(input_str, choices):
  matches = process.extract(input_str, choices, scorer=fuzz.token_sort_ratio)
  best_match = max(matches, key=lambda x: x[1])

  if best_match[1] >= 20:
    return best_match[0]
  else:
    return None


In [ ]:
input_features = []
for f in feature_names:
  features_from_data = feature_label_match(f, cols)
  input_features.append(features_from_data)

# list comprehension:
# input_features = [feature_label_match(f, cols) for f in feature_names]
input_labels = [feature_label_match(l, cols) for l in label_names]
# print(input_features)
# print(input_labels)

In [ ]:
categorical_labels = data[input_labels[0]].unique()

In [ ]:
# label_encoder = LabelEncoder()
# for idx in range(len(input_features)):
#   # data[input_features[idx]].values returns a list of values in the given column
#   if data[input_features[idx]].values.dtype == object:
#     data[input_features[idx]] = label_encoder.fit_transform(data[input_features[idx]])
# print(data[input_features[idx]])

In [ ]:
print(labels)

Suicide Committed


In [ ]:
# Handling of missing data:

# CREATE HUGE FUNCTION FOR DATA PREPROCESSING THAT TAKES IN TRAINING AND TESTING DATA SEPARATELY:

# features = data.drop(input_labels, axis=1)
# labels = data.drop(input_features, axis=1)

# X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

def convert_to_numerical(data, input_names):
    label_encoder = LabelEncoder()
    onehotencoder = OneHotEncoder()

    encoded_features = []
    object_features = []
    object_labels = []
    for name in range(len(input_names)):
      # data[input_names[name]] is a pandas series
        if data[input_names[name]].values.dtype == object:
            if input_names == input_features:
              object_features.append(input_names[name])
            else:
              object_labels.append(input_names[name])
            # print(f"the data type of the values of feature/label #{name} is categorical. We must change it to numerical before using it as input to our model")
            # label_encoder.fit_transform() takes these data types: 1-dimensional array-like input, such as a list, NumPy array, or Pandas Series
            # label_encoder.fit_transform() returns an object that is the same data type as the input given. In this case, it returns a Pandas Series
            data[input_names[name]] = label_encoder.fit_transform(data[input_names[name]])

    # print(input_labels)
    # print(input_features)
    return object_features, object_labels

def preprocess_two(data, input_features, input_labels, source):
  data_types = data.dtypes
  numerical_columns = []
  categorical_columns = []
  for column, dtype in data_types.items():
    if dtype == 'int64' or dtype == 'float64':
      numerical_columns.append(column)
    else:
      categorical_columns.append(column)

  # Handling of missing numerical data:
  if numerical_columns:
    imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    imputer = imputer.fit(data[numerical_columns])
    data[numerical_columns] = imputer.transform(data[numerical_columns])
  # Handling of missing categorical data:
  elif categorical_columns:
    imputer = SimpleImputer(strategy='most_frequent')
    imputer = imputer.fit(data[categorical_columns])
    # most_frequent_values = imputer.statistics_
    # print(most_frequent_values)
    data[categorical_columns] = imputer.transform(data[categorical_columns])


  if source == 'X_train' or source == 'X_test':
    object_features, object_labels = convert_to_numerical(data, input_features)
    returned_data = pd.get_dummies(data)
    # returned_data = pd.get_dummies(data.drop(input_labels, axis=1), columns=object_features)
  else:
    object_features, object_labels = convert_to_numerical(data, input_labels)
    # returned_data = pd.get_dummies(data.drop(input_features, axis=1), columns=object_labels)
    returned_data = pd.get_dummies(data)
    returned_labels = pd.get_dummies(data)


  # SCALE DATA:
  min_max_scaler = MinMaxScaler(feature_range=(-1,1))
  scaled_data = min_max_scaler.fit_transform(returned_data)

  # Normalizing data such that featues have a zero mean and std dev of 1:
  scaler = StandardScaler()
  scaled_data = scaler.fit_transform(scaled_data)

  # scaler.fit_transform() takes as a 2-d array-like oject as input, such as numpy arrays, datframes
  # 'scaler.fit_transform()' returns a numpy array
  return scaled_data

def preprocess(file_path, features, labels):
    # Mapping of file extensions to read functions
    read_functions = {
        'csv': pd.read_csv,
        'xlsx': pd.read_excel,
        'xls': pd.read_excel,
        'json': pd.read_json,
        'txt': pd.read_csv,  # Example: plain text file
        'h5': pd.read_hdf,   # Example: HDF5 file
        'parquet': pd.read_parquet  # Example: Parquet file
        # Add more file formats and read functions as needed
    }

    file_extension = file_path.split('.')[-1].lower()

    if file_extension in read_functions:
        read_function = read_functions[file_extension]
        data = read_function(file_path)
    else:
        raise ValueError(f"File extension not given or an unsupported file format: '{file_extension}' has been provided")

    col_names = data.columns.tolist()
    print(col_names)
    feature_names = features.strip().split(',')
    label_names = labels.strip().split(',')

    # Perform fuzzy matching to find the best matches for features and labels
    # 'input_features' is a list of avaiable features the model will use
    input_features = [feature_label_match(f,col_names) for f in feature_names]
    input_labels = [feature_label_match(l, col_names) for l in label_names]


    features = data.drop(input_labels, axis=1)
    labels = data.drop(input_features, axis=1)

    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

    X_train_preprocessed = preprocess_two(X_train, input_features, input_labels, source='X_train')
    X_test_preprocessed = preprocess_two(X_test, input_features, input_labels, source='X_test')
    y_train_preprocessed = preprocess_two(y_train, input_features, input_labels, source='y_train')
    y_test_preprocessed = preprocess_two(y_test, input_features, input_labels, source='y_test')

    return X_train_preprocessed, X_test_preprocessed, y_train_preprocessed, y_test_preprocessed


In [ ]:
def preprocess(file_path, features, labels):
    # Mapping of file extensions to read functions
    read_functions = {
        'csv': pd.read_csv,
        'xlsx': pd.read_excel,
        'xls': pd.read_excel,
        'json': pd.read_json,
        'txt': pd.read_csv,  # Example: plain text file
        'h5': pd.read_hdf,   # Example: HDF5 file
        'parquet': pd.read_parquet  # Example: Parquet file
        # Add more file formats and read functions as needed
    }

    file_extension = file_path.split('.')[-1].lower()

    if file_extension in read_functions:
        # read_function = pd.read_csv
        read_function = read_functions[file_extension]
        # data = pd.read_csv(file_path)
        data = read_function(file_path)
    else:
        raise ValueError(f"File extension not given or an unsupported file format: '{file_extension}' has been provided")

    return data


In [ ]:
read_functions = {
    'csv': pd.read_csv,
    'xlsx': pd.read_excel,
    'xls': pd.read_excel,
    'json': pd.read_json,
    'txt': pd.read_csv,  # Example: plain text file
    'h5': pd.read_hdf,   # Example: HDF5 file
    'parquet': pd.read_parquet  # Example: Parquet file
    # Add more file formats and read functions as needed
}

df = pd.read_excel
print(df)

<function read_excel at 0x7f5ac3579fc0>


In [ ]:
df = preprocess('inmate_history.csv', features, labels)
print(df)

<function read_csv at 0x7f5ac3578e50>
    Meals Missed (last 30 days)  Violent Offenses (last 30 days)  \
0                            10                               10   
1                            28                               28   
2                            25                               27   
3                             0                                9   
4                            17                               23   
..                          ...                              ...   
95                           13                                1   
96                           30                               29   
97                           26                               27   
98                           21                               28   
99                            1                                7   

    Medication Refused (last 30 days) History of Depression  \
0                                  12                   yes   
1                  

In [ ]:
X_train_preprocessed, X_test_preprocessed, y_train_preprocessed, y_test_preprocessed = preprocess('inmate_history.csv', features, labels)
print(X_train_preprocessed)

In [ ]:
label_encoder = LabelEncoder()
onehotencoder = OneHotEncoder()

encoded_features = []
object_features = []
object_labels = []

def convert_to_numerical(data, input_names):
    for name in range(len(input_names)):
      # data[input_names[name]] is a pandas series
        if data[input_names[name]].values.dtype == object:
            if input_names == input_features:
              object_features.append(input_names[name])
            else:
              object_labels.append(input_names[name])
            # print(f"the data type of the values of feature/label #{name} is categorical. We must change it to numerical before using it as input to our model")
            # label_encoder.fit_transform() takes these data types: 1-dimensional array-like input, such as a list, NumPy array, or Pandas Series
            # label_encoder.fit_transform() returns an object that is the same data type as the input given. In this case, it returns a Pandas Series
            data[input_names[name]] = label_encoder.fit_transform(data[input_names[name]])



convert_to_numerical(data, input_features)
convert_to_numerical(data, input_labels)

# one-hot encoding:
# each label is transformed into a row of zeros except for the index corresponding to the label, which is set to 1
returned_features = pd.get_dummies(data.drop(input_labels, axis=1), columns=object_features)
returned_labels = pd.get_dummies(data.drop(input_features, axis=1), columns=object_labels)



# LABEL_ENCODER VS TO_CATEGORICAL:
# Use 'label_encoder' for features (numerical encoding)
# Use 'label_encoder' followed by 'to_categorical' for labels. Just using 'label_encoder' ensures that each class is represented by a unique integer value.
# However, numerical encoding introduces ordinal relationship between the classes, which might not be appropriate for certain classification tasks.
# 'to_categorical' performs one-hot encoding, which represents each class as a binary vector
# In this specific instance, there is not importance of order for the labels 'yes' and 'no'. This is because 'yes' and 'no' are not directly related to one another
# If the classes were instead 'low', 'medium', 'high', then only performing numerical encoding might allow the model to learn the ordinal relationship between these labels/classes


In [ ]:
# returned_labels = data[input_labels]
# 'to_categorical' takes an array-like object as input, such as a numpy array or a pandas series
# 'to_categorical' returns a numpy array
# returned_labels = to_categorical(returned_labels)

# Difference between scaling and Normalization:
# Scaling (transforming data into a (0,1) or (-1,1) range). Ypu would typically use MinMaxScaler() to do this
# Normalization (transofmrs data into having a zero mean and std dev. of 1. The range of the transformed data does not have to be between 0 and 1)

def scale_normalize_data(data):
  # Scaling data into range (-1,1):
  min_max_scaler = MinMaxScaler(feature_range=(-1,1))
  scaled_features = min_max_scaler.fit_transform(data)

  # Normalizing data such that featues have a zero mean and std dev of 1:
  scaler = StandardScaler()
  scaled_features = scaler.fit_transform(scaled_features)

  # scaler.fit_transform() takes as a 2-d array-like oject as input, such as numpy arrays, datframes
  # 'scaler.fit_transform()' returns a numpy array
  return scaled_features

scaled_features = scale_normalize_data(returned_features)
print(scaled_features)


In [ ]:
print(labels)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(scaled_features, returned_labels, test_size=0.2, random_state=42)

model = Sequential()
model.add(Input(shape=X_train_preprocessed.shape[1]))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(returned_labels.shape[1], activation='softmax'))

checkpoint = ModelCheckpoint('weights_best.hdf5', save_best_only=True)
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_preprocessed, y_train_preprocessed, epochs=10, batch_size=32, validation_data=(X_test_preprocessed, y_test_preprocessed), callbacks=[checkpoint, early_stopping])

model.load_weights('weights_best.hdf5')

model.save("user_model.h5")

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test)
print(accuracy)



In [ ]:
print(X_train.shape) # returns (80% of the num of subjects (rows), the num of features)
print(y_train.shape) # returns (80% of the num of subjects (rows), the num of classes)
print(X_test.shape) # returns (20% of the num of subjects (rows), the num of features)
print(y_test.shape) # returns (20% of the num of subjects (rows), the num of classes)
print(returned_labels.shape)

In [ ]:
histo1 = data.hist(data.columns)
print(histo1)

#x axis of histo1 represents the length (in pixels) of the Major Axis of the bean
#y axis of histo1 represents the count of that particular length. For example, histo1 shows that there is a peak of 4800 for a length of 250